Some other additional stuff

In [1]:
import sys
import pandas as pd
import numpy as np
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler


In [2]:
df_train = pd.read_csv("../input/clean_train.csv", low_memory=False)

In [3]:
requester = pd.read_csv("../input/requester.csv", sep=';')

In [4]:
df_train['win_count'].fillna(df_train['win_count'].mean(), inplace=True)

In [5]:
df_train = pd.merge(df_train, requester[['id', 'requester_type', 'gender', 'birth_year']], how='left', left_on = 'requester', right_on = 'id')

In [6]:
df_train['gender'].fillna(1, inplace=True)

In [7]:
df_train['birth_year'].fillna(df_train['birth_year'].mean(), inplace=True)

In [8]:
df_train.drop(['order_number','id'], axis=1, inplace=True)

In [9]:
def get_age(x):
    return 2020 - x['birth_year']

In [10]:
df_train['age'] = df_train.apply(get_age, axis=1)

In [11]:
df_train.drop(['birth_year'], axis=1, inplace=True)

In [12]:
df_train['gender'] = df_train['gender'].astype('int64')

In [13]:
order_count = df_train[['requester', 'service']].groupby(['requester']).count()

In [14]:
order_count.rename({'service':'order_count'}, axis=1, inplace=True)

In [15]:
order_count.reset_index(inplace=True)

In [16]:
df_train = pd.merge(df_train, order_count, how='left', on = 'requester')

In [17]:
df_train['month'] = pd.to_datetime(df_train['order_date']).dt.month

In [18]:
df_train['week'] = pd.to_datetime(df_train['order_date']).dt.week

In [19]:
df_train['year'] = pd.to_datetime(df_train['order_date']).dt.year

In [20]:
pop_csi = list(df_train['custom_service_id'].value_counts()[:50].index)

In [21]:
def change_csi(x):
    if x['custom_service_id'] in pop_csi:
        return x['custom_service_id']
    return 777

In [22]:
df_train['custom_service_id'] = df_train.apply(change_csi, axis=1)

In [23]:
df_train.tail()

,order_date,requester,service,service_title,mfc,internal_status,external_status,order_type,department_id,custom_service_id,...,legal,auto_ping_queue,win_count,requester_type,gender,age,order_count,month,week,year
2713442,2020-05-18 19:06:26.653,177596976,43,1275,4396466,2,2,SERVICE,12870937,210963,...,False,True,14.0,1,2,22.0,8,5,21,2020
2713443,2020-05-18 19:07:50.170,3663125,43,98,1956656,2,2,SERVICE,316134,48333290,...,False,True,13.0,1,2,45.0,7,5,21,2020
2713444,2020-05-18 19:26:17.127,86143570,43,98,19146184,2,2,SERVICE,316134,48333290,...,False,True,3.0,1,2,62.0,2,5,21,2020
2713445,2020-05-18 19:32:53.107,138811055,43,491,1956423,10,5,SERVICE,316160,210889936,...,False,True,6.0,1,1,46.0,3,5,21,2020
2713446,2020-05-18 19:35:25.083,138811055,43,724,1956423,10,5,SERVICE,316160,210890128,...,False,True,6.0,1,1,46.0,3,5,21,2020


In [24]:
person_data = df_train[['requester', 'requester_type', 'gender', 'age', 'order_count']].groupby(['requester']).min()

In [25]:
person_data.reset_index(inplace=True)

In [26]:
#grouped = df_train.loc[df_train['order_count'] <= 6].groupby('requester')

In [27]:
grouped = df_train.groupby('requester')

In [28]:
columns = ['requester', 
 'order_date',
 'service',
 'service_title',
 'mfc',
 'internal_status',
 'external_status',
 'order_type',
 'department_id',
 'custom_service_id',
 'service_level',
 'is_subdep',
 'is_csid',
 'proc_time',
 'dayofweek',
 'day_part',
 'month',
 'week',
 'year',
 'person',
 'sole',
 'legal',
 'auto_ping_queue',
 'win_count']

In [29]:
dt = df_train[columns].set_index(['requester', 'order_date'])
dt = dt.sort_index(axis = 0)

In [30]:

dt1 = dt.groupby('requester').shift(1)
dt2 = dt.groupby('requester').shift(2)
dt3 = dt.groupby('requester').shift(3)


# merge dataframes

newdf = pd.merge(dt, dt1, how = 'left', on = ['requester', 'order_date'], sort = False, suffixes = ('', '_1'))
newdf = pd.merge(newdf, dt2, how = 'left', on = ['requester', 'order_date'], sort = False, suffixes = ('', '_2'))
newdf = pd.merge(newdf, dt3, how = 'left', on = ['requester', 'order_date'], sort = False, suffixes = ('', '_3'))

In [31]:
final = newdf.loc[newdf['service_title_1'].notnull()].reset_index().drop(['order_date'], axis=1)

In [32]:
final = pd.merge(final, person_data, how='left', on = 'requester')

In [33]:
final.to_csv('train_3lags_v3.csv', index=False)